In [ ]:
!cp '/content/drive/My Drive/TCC_data/audios.zip' .
!unzip audios.zip

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import warnings

In [ ]:
plt.style.use('classic')
display.set_matplotlib_formats('jpg', quality=94)
warnings.filterwarnings('ignore')

## Compare speech recognition

In [ ]:
!pip install -U pydub

In [ ]:
import json
import requests
import io
from pydub import AudioSegment

def split_into_chunks(
    segment, length=20000 / 1001, split_on_silence=False, noise_threshold=-36
):
    chunks = list()

    if split_on_silence is False:
        for i in range(0, len(segment), int(length * 1000)):
            chunks.append(segment[i:i + int(length * 1000)])
    else:
        while len(chunks) < 1:
            chunks = split_on_silence(segment, noise_threshold)
            noise_threshold += 4

    for i, chunk in enumerate(chunks):
        if len(chunk) > int(length * 1000):
            subchunks = split_into_chunks(
                chunk, length, split_on_silence, noise_threshold + 4
            )
            chunks = chunks[:i - 1] + subchunks + chunks[i + 1:]

    return chunks


def preprocess_audio(audio):
    return audio.set_sample_width(2).set_channels(1).set_frame_rate(22500)


def read_audio_into_chunks(file_path):
    audio = AudioSegment.from_file(file_path)
    return split_into_chunks(preprocess_audio(audio))


def transcribe_audio_wit(file_path):
    url = 'https://api.wit.ai/speech'

    authorization = 'Bearer ' + 'EWIVZ2MWYZQ7EPJYFHW27MPFWMKP35ZH'
    content_type = 'audio/raw;' \
        'encoding=signed-integer;' \
        'bits=16;' \
        'rate=22500;' \
        'endian=little'

    # defining headers for HTTP request
    headers = {
        'authorization': authorization,
        'content-type': content_type
    }

    chunks = read_audio_into_chunks(file_path)

    text = []
    for audio in chunks:
        response = requests.post(
            url,
            headers=headers,
            data=io.BufferedReader(io.BytesIO(audio.raw_data))
        )

        try:
            # Get the text
            data = json.loads(response.content)
            if 'text' in data:
                text.append(data['text'])
        except:
            pass

    return ' '.join(text)

In [ ]:
sentences = [
    "A inauguração da vila é quarta ou quinta feira",
    "Vote se você tiver o título de eleitor",
    "Hoje é fundamental encontrar a razão da existência humana",
    "A temperatura é mais amena a noite",
    "Em muitas cidades a população está diminuindo",
    "Nunca se deve ficar em cima do morro",
    "Para as pessoas estranhas, o panorama é desolador",
    "E bom te ver colhendo flores menino",
    "Eu finjo me banhar num lago ao amanhecer",
    "Sua sensibilidade mostrará o caminho",
    "A Amazônia é a reserva ecológica do globo",
    "O ministério mudou demais com a eleição",
    "Nova metas surgem na informática",
    "O capital de uma empresa depende de sua produção",
    "Se não fosse ela tudo teria sido melhor",
    "A principal personagem do filme é uma gueixa",
    "Espere seu amigo em casa",
    "A juventude tinha que revolucionar a escola",
    "A cantora terá quatro meses para ensaiar seu canto",
    "Esse tema foi falado no congresso"
]

In [ ]:
!cp -R '/content/drive/My Drive/TCC_data/embeddings/' .

!unzip -d 'word2vec' "embeddings/word2vec_*.zip"

In [ ]:
from gensim import corpora
from gensim.matutils import softcossim
from gensim.models import KeyedVectors

emb_model = KeyedVectors.load_word2vec_format('word2vec/cbow_s50.txt')

def cosine_similarity(reference, hypothesis):
    reference = reference.split()
    hypotesis = hypothesis.split()
    documents = [hypotesis, reference]
    dictionary = corpora.Dictionary(documents)

    similarity_matrix = emb_model.similarity_matrix(dictionary)

    hypotesis = dictionary.doc2bow(hypotesis)
    reference = dictionary.doc2bow(reference)

    return softcossim(hypotesis, reference, similarity_matrix)

In [ ]:
from collections import defaultdict

d = defaultdict(list)

In [ ]:
for i in range(1, 21):
    sent = sentences[i - 1]
    f = 'ground_truth/{}.wav'.format(i)
    orig = transcribe_audio_wit(f)
    d['ground_truth'].append(cosine_similarity(sent, orig))

    print(sent, f'({orig})')
    display.display(display.Audio(f))
    for m in ['dctts', 'tacotron', 'mozilla_tts']:
        f = '{}/{}.wav'.format(m, i)
        trans = transcribe_audio_wit(f)
        d[m].append(cosine_similarity(sent, trans))
    
        print(m, ':' , trans)
        display.display(display.Audio(f))
    print('*' * 80)

In [ ]:
scores = {}
for m in d:
    scores[m] = np.mean(d[m])
scores